# Onclusive Machine Learning Challenge
## Build an ML system to verify the veracity of claims in dataset PUBHEALTH.
### Outlining the steps and demonstrate the performance of the final model.  

In [1]:
from datasets import load_dataset
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

In [2]:
dataset = load_dataset("health_fact")

Using custom data configuration default
Reusing dataset health_fact (C:\Users\david\.cache\huggingface\datasets\health_fact\default\1.1.0\99503637e4255bd805f84d57031c18fe4dd88298f00299d56c94fc59ed68ec19)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("nbroad/longformer-base-health-fact")
model = AutoModelForSequenceClassification.from_pretrained("nbroad/longformer-base-health-fact")

In [4]:
cls = pipeline(task="text-classification", model=model, tokenizer=tokenizer)

In [7]:
tabl = cls(dataset['train']['claim'][:100])